# Part I. ETL Pipeline for Pre-Processing the Files

Import Python packages

In [37]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

Creating list of filepaths to process original event csv data files

In [38]:
# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [39]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if row[0] == '':
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [40]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra coding portion of your project.

Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns:
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Creating a Cluster

In [41]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

## Create Keyspace

In [42]:
try:
    session.execute("""DROP KEYSPACE IF EXISTS keyspace_name""")

    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkifydb
    WITH REPLICATION =
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
                    )
except Exception as e:
    print(e)

## Set Keyspace

In [43]:
try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print(e)

## Query 1

*Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4*

Create the table to answer query 1.
- Columns: `sessionId`, `itemInSession`, `artist`, `song`, `length`
- Primary key: `sessionId` and `itemInSession`

### Table creation

In [44]:
query = ("""
CREATE TABLE IF NOT EXISTS reproductions (
sessionId int,
itemInSession int,
artist text,
song text,
length float,
PRIMARY KEY (sessionId, itemInSession));
""")
try:
    session.execute(query)
except Exception as e:
    print(e)

### Load the data

In [45]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO reproductions (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

### Run select statement

In [46]:
query = "select * from reproductions WHERE sessionId = 338 and itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


## Query 2

*Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182*

Create the table to answer query 1.
- Columns: `sessionId`, `itemInSession`, `userid`, `artist`, `song`, `firstName`, `lastName`
- Primary key: `userid`, `sessionId` and `itemInSession`


### Table Creation

In [47]:
query = ("""
CREATE TABLE IF NOT EXISTS user_activitiy (
session_id int,
item_session int,
user_id int,
artist text,
song text,
first_name text,
last_name text,
PRIMARY KEY ((user_id, session_id), item_session)
);
""")
try:
    session.execute(query)
except Exception as e:
    print(e)

### Load the data

In [48]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_activitiy (session_id, item_session, user_id, artist, song, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), int(line[10]), line[0], line[9], line[1], line[4]))

### Run selection statement

In [49]:
query = "select artist, song, first_name, last_name from user_activitiy WHERE user_id = 10 and session_id = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row)

Row(artist='Down To The Bone', song="Keep On Keepin' On", first_name='Sylvie', last_name='Cruz')
Row(artist='Three Drives', song='Greece 2000', first_name='Sylvie', last_name='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', first_name='Sylvie', last_name='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', first_name='Sylvie', last_name='Cruz')


## Query 3

*Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'*

Create the table to answer query 1.
- Columns: `sessionId`, `itemInSession`, `song`, `firstName`, `lastName`
- Primary key: `song`, `sessionId` and `itemInSession`

### Table Creation

In [50]:
query = ("""
CREATE TABLE IF NOT EXISTS listened_songs (
session_id int,
item_session int,
song text,
first_name text,
last_name text,
PRIMARY KEY ((song), session_id, item_session)
);
""")
try:
    session.execute(query)
except Exception as e:
    print(e)

### Load the data

In [51]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO listened_songs (session_id, item_session, song, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[9], line[1], line[4]))

### Run select statement

In [52]:
query = "select first_name, last_name from listened_songs WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row)

Row(first_name='Sara', last_name='Johnson')
Row(first_name='Jacqueline', last_name='Lynch')
Row(first_name='Tegan', last_name='Levine')


## Close the session and cluster connection

In [53]:
session.shutdown()
cluster.shutdown()